# Validate, standardize & annotate

We'll walk you through the following flow: 
1. define validation criteria
2. validate & standardize metadata
3. save validated & annotated artifacts

:::{dropdown} How do we validate metadata?

Registries in your database define the "truth" for metadata.

For instance, if "Experiment 1" has been registered as the `name` of a `ULabel` record, it is a validated value for field `ULabel.name`.

:::

```{toctree}
:maxdepth: 1
:hidden:

can-validate
annotate-for-developers
```

Install the `lamindb` Python package:
```shell
pip install 'lamindb[bionty]'
```

In [ ]:
!lamin init --storage ./test-annotate --schema bionty

In [ ]:
import lamindb as ln
import bionty as bt
import pandas as pd
import anndata as ad

ln.settings.verbosity = "hint"

Let's start with a DataFrame that we'd like to validate:

In [ ]:
df = pd.DataFrame({
    "temperature": [37.2, 36.3, 38.2],
    "cell_type": ["cerebral pyramidal neuron", "astrocyte", "oligodendrocyte"],
    "assay_ontology_id": ["EFO:0008913", "EFO:0008913", "EFO:0008913"],
    "donor": ["D0001", "D0002", "DOOO3"],
})
df

## Validate and standardize metadata

In [ ]:
# define validation criteria for the categoricals
categoricals = {
    "cell_type": bt.CellType.name,
    "assay_ontology_id": bt.ExperimentalFactor.ontology_id,
    "donor": ln.ULabel.name,
}
# create an object to guide validation and annotation
annotate = ln.Annotate.from_df(df, categoricals=categoricals)
# validate
validated = annotate.validate()
validated

## Validate using registries in another instance

Sometimes you want to validate against other existing registries, for instance [cellxgene](https://lamin.ai/laminlabs/cellxgene).

This allows us to directly transfer values that are currently missing in our registries from the [cellxgene instance](https://lamin.ai/laminlabs/cellxgene).

In [ ]:
annotate = ln.Annotate.from_df(
    df, 
    categoricals=categoricals,
    using="laminlabs/cellxgene",  # pass the instance slug
)
annotate.validate()

## Register new metadata labels

Our current database instance is empty. Once you populated its registries, saving new labels will only rarely be needed. You'll mostly use your lamindb instance to validate any incoming new data and annotate it.

In [ ]:
annotate.add_validated_from(df.cell_type.name)

In [ ]:
# use a lookup object to get the correct spelling of categories from public reference
# pass "public" to use the public reference
lookup = annotate.lookup()
lookup

In [ ]:
cell_types = lookup[df.cell_type.name]

In [ ]:
cell_types.cerebral_cortex_pyramidal_neuron

In [ ]:
# fix the typo
df.cell_type = df.cell_type.replace({"cerebral pyramidal neuron": cell_types.cerebral_cortex_pyramidal_neuron.name})

annotate.add_validated_from(df.cell_type.name)

In [ ]:
# register non-validated terms
annotate.add_new_from(df.donor.name)

In [ ]:
# validate again
validated = annotate.validate()
validated

## Validate an AnnData object

Here we specify which `var_fields` and `obs_fields` to validate against.

In [ ]:
df.index = ["obs1", "obs2", "obs3"]

X = pd.DataFrame({"TCF7": [1, 2, 3], "PDCD1": [4, 5, 6], "CD3E": [7, 8, 9], "CD4": [10, 11, 12], "CD8A": [13, 14, 15]}, index=["obs1", "obs2", "obs3"])

adata = ad.AnnData(X=X, obs=df)
adata

In [ ]:
annotate = ln.Annotate.from_anndata(
    adata, 
    var_index=bt.Gene.symbol,
    categoricals=categoricals, 
    organism="human",
)

In [ ]:
annotate.validate()

In [ ]:
annotate.add_validated_from("all")

In [ ]:
annotate.validate()

## Save an artifact

The validated object can be subsequently saved as an {class}`~lamindb.Artifact`:

In [ ]:
artifact = annotate.save_artifact(description="test AnnData")

In [ ]:
artifact.describe()

## Save a collection

Register a new collection for the registered artifact:

In [ ]:
# register a new collection
collection = annotate.save_collection(
    artifact,  # registered artifact above, can also pass a list of artifacts
    name="Experiment X in brain",  # title of the publication
    description="10.1126/science.xxxxx",  # DOI of the publication
    reference="E-MTAB-xxxxx", # accession number (e.g. GSE#, E-MTAB#, etc.)
    reference_type="ArrayExpress" # source type (e.g. GEO, ArrayExpress, SRA, etc.)
)

In [ ]:
collection.artifacts.df()